In [1]:
import qminospy
from qminospy.me2 import ME_NLP
import cobra
import cobrame
import pickle

/home/jt/work/cobrapy-0.5.11/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/jt/work/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


## Load Mother ME-model

In [2]:
with open('./me_models/solution_mother.pickle', 'rb') as f:
    mother = pickle.load(f)

## Load Spore ME-model

In [3]:
with open('./me_models/solution_spore.pickle', 'rb') as f:
    spore = pickle.load(f)

## Load ME-model

In [4]:
with open('./me_models/solution.pickle', 'rb') as f:
    me = pickle.load(f)

## Solving a ME-model

In [5]:
def solve_me_model(me, max_mu=1., precision=1e-6, min_mu=0, using_soplex=True,
                  compiled_expressions=None, verbosity = 2, mu_fix = False,
                  growth_key='mu'):
    from qminospy.me1 import ME_NLP1
    ## If fixed growth rate, solve as LP
    if mu_fix:
        me_nlp = ME_NLP1(me)
        me_nlp.solvelp(mu_fix)
    else:
        ## 
        if using_soplex:
            from cobrame.solve.algorithms import binary_search
            binary_search(me, min_mu=min_mu, max_mu=max_mu, debug=True, mu_accuracy=precision,
                compiled_expressions=compiled_expressions)
        else:
            # The object containing solveME methods--composite that uses a ME model object 
            me_nlp = ME_NLP1(me, growth_key=growth_key)
            # Use bisection for now (until the NLP formulation is worked out)
            muopt, hs, xopt, cache = me_nlp.bisectmu(precision=precision, mumax=max_mu, verbosity=verbosity)
            try:
                me.solution.f = me.solution.x_dict['biomass_dilution']
            except:
                pass


In [ ]:
solve_me_model(me, max_mu = 0.5, min_mu = .05, using_soplex=False, precision = 1e-2)

iter	muopt    	a     	b     	mu1       	stat1
Finished compiling expressions in 30.479065 seconds
Finished substituting S,lb,ub in 1.065972 seconds
Finished makeME_LP in 0.152913 seconds
Getting MINOS parameters from ME_NLP...
1 0.0 0.0 0.25 0.25 1
